In [12]:
# Importacion de librerias
# from google.colab import drive    # Para usar archivos de drive
import nltk # Para NLP
import re     # Manejo de expresiones regulares
import numpy as np    # Para manejo de arreglosm
from sklearn.feature_extraction.text import CountVectorizer     # Para vectorizar el contexto
from sklearn.feature_extraction.text import TfidfTransformer    # Para vectorizar el contexto con IDF

# Descargamos recurso para incluir word_tokenizer
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\reini\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\reini\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [13]:
# Manejo de Archivos

# Apertura de archivos
def openA(rutaArchivo):
  return open(rutaArchivo, 'r', encoding='utf-8')

# Escritura de archivos
def writeA(rutaArchivo, data):
  file = open(rutaArchivo,"w", encoding='utf-8')
  file.write(str(data))
  file.close()

# Lectura del archivo
def readA(archivo):
  contenido = archivo.read()
  return contenido

In [14]:
# Preprocesamiento

# Limpieza del archivo
def clean(plainText):
  # Expresion regular para eliminar los xml tags y saltos de linea
  regex_xml = r'<[^>]+>'

  # Quitando del texto los xml tags
  texto = re.sub(regex_xml, '', plainText)

  # Pasando a lowercase
  texto = texto.lower()

  return texto


# Tokenizacion
def tokenizacion(texto):
  tokens = nltk.word_tokenize(texto)
  return tokens

# Limpieza de tokens, Eliminar todos los tokens que no son palabras (Todo lo que no sea letra del alfabeto)
def cleanTokens(tokens):
  tokens = [token for token in tokens if token.isalpha()]
  return tokens

# Eliminacion de stopwords
def deleteStopwords(tokens):
  return tokens

# Lematización del texto
def lemaText(tokens):
  return tokens

# Preprocesamiento del texto
def preproText(plainText):
  text = clean(plainText)
  tokens = tokenizacion(text)
  tokens = cleanTokens(tokens)
  return tokens

# Parseador de texto a Text
def parseText(tokens):
  # NOTA: Para obtener la frecuencia de distribución primero debemos parsear el texto con Text
  text = nltk.Text(tokens)
  return text

# Generar el vocabulario
def getVocabulary(tokens, text):
  fdist = nltk.FreqDist(text)          # Da el total de palabras que se han contado
  vocabulary = fdist.keys()            # Da una lista de todos los tipos distintos de palabras en el texto
  vocabulary = sorted(vocabulary)      # Ordanamiento alfabeticamente

  return vocabulary

In [15]:
# Codificacion en espacio vectorial con bolsa de palabras

"""
Función que obtiene los contextos de una palabra
@param texto: Son los tokens parseados a Texto
@param word: Es la palabra a buscar el contexto (En este caso cada una del vocabulario)
@param w: Es la ventana con la que se trabajara (En este caso es de 8 (4 para izquierda - 4 para derecha))
"""
def getContextCollection(texto, word, w):
  wLeft = wRight = w//2
  contexto_left = []
  contexto_right = []
  contextoW = {}
  aux = []
  print("Tipo de texto: ", type(texto))
  # Obteniendo la concordancia (se imprime)
  # texto.concordance(word)

  # Contamos la candidad de concordancia
  cantidadConcordancias = texto.count(word)

  for concordancia in texto.concordance_list(word):  # Ejemplo de documentacion de concordance: https://www.nltk.org/howto/concordance.html#concordance-list
        # Creacion de AUX para conocer la palabra anterior
        aux.append(word)
        # La línea de la concordancia contiene la palabra objetivo y su contexto
        linea = concordancia.line
        tokens_linea = linea.split()

        # Buscamos la posición de la palabra brindada en la línea
        indice = tokens_linea.index(word)

        # Extraemos los 4 tokens a la izquierda y derecha
        izquierda = tokens_linea[max(0, indice-wLeft):indice]
        derecha = tokens_linea[(indice+1):max(0, (indice+wRight+1))]

        # Guardamos los tokens en un arreglo
        if len(aux) == 1:
          contexto_left = izquierda
          contexto_right = derecha

        elif aux[-2] == word:
          contexto_left = contexto_left + izquierda
          contexto_right = contexto_right + derecha

        # Unimos los contextos
        contextoW = contexto_left + contexto_right
  return contexto_left, contexto_right, contextoW

# Creacion de lista de contextos ([contexto_w1, contexto_w2,...,contexto_wn])
def getBagWords(text, vocabulary):
  contextos = []
  for word in vocabulary:
      _, _, contexto = getContextCollection(text, word, 8)
      contextos.append(contexto)
  return contextos

# Creacion de vetor con valores de Document Frecuency
def getDocumentF(bagWords):
  idf = []
  for vectorTerm in bagWords:
    count = 0
    for context in vectorTerm:
      if context != 0:
        count = count + 1
    idf.append(count)

  return idf


###############################################################################################################################################################################
# Vectorizacion
def vectorizacion(bagWords, lenVocabulary, typeV):
  # Convertimos en una sola cadena (separada por espacios) cada uno de los contextos (documentos)
  documentos = [' '.join(doc) for doc in bagWords]

  # Inicializamos CountVectorizer
  vectorizer = CountVectorizer() # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

  # Vectorizamos los documentos
  vectores = vectorizer.fit_transform(documentos)

  matrizF = vectores.toarray() # Convierte vectores a una matriz densa
  # Tranformacion de la matriz segun el tipo de vectorizacion
  if typeV == 1: # Count
    matrizF = matrizF
  elif typeV == 2: # Frecuencia Relativa
    matrizF = matrizF / np.sum(matrizF, axis=1).reshape(-1, 1)
  elif typeV == 3: # TF con log (Transformacion sublineal)
    matrizF = np.log(1 + matrizF)
  elif typeV == 4: # IDF
    # k = np.array(getDocumentF(bagWords))
    # print(k)
    # for i in range(lenVocabulary):
    #   matrizF[i] = float(np.log(lenVocabulary + 1 / k[i]))
    # print(matrizF)
    idf = TfidfTransformer(use_idf=True, smooth_idf=False, sublinear_tf=False)
    matrizF_idf = idf.fit_transform(matrizF)
    matrizF = matrizF_idf.toarray()

  # Obteniendo los nombres de cada vector
  namesVectors = vectorizer.get_feature_names_out()

  # Creando diccionario de palabra con su vector
  dicVectorWord = {}
  for i in range(lenVocabulary):
    dicVectorWord[namesVectors[i]] = matrizF[i].tolist()

  return dicVectorWord


In [16]:
# Mediciones de similitud
"""
Funcion para obtener el coseno entre 2 vectores
"""
def getCos(x , y): # xy / |x||y|
  pDot = np.dot(x,y)
  normaX = np.linalg.norm(x)
  normaY = np.linalg.norm(y)
  pNorma = np.dot(normaX, normaY)
  cos = pDot / pNorma

  return round(cos, 3)

In [17]:
# Creacion de diccionario de similitudes
def getSimilitud(word, dicVecWord):
  similitud = {}
  for doc in dicVecWord:
    cosenoT = getCos(dicVecWord[word], dicVecWord[doc])
    similitud[doc] = float(cosenoT)

  similitud = dict(sorted(similitud.items(), key=lambda item: item[1], reverse=True))

  return similitud

In [18]:
# Main
# rutaArchivo = "/content/drive/MyDrive/PLN/Textos/e990505_mod.htm"
rutaArchivo = "./Textos/e990505_mod.htm"

archivo = openA(rutaArchivo)
plainText = readA(archivo)
tokens = preproText(plainText)
text = parseText(tokens)
vocabulary = getVocabulary(tokens, text)
bagOfWords = getBagWords(text, vocabulary)


Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  <class 'nltk.text.Text'>
Tipo de texto:  

In [8]:
dicVectorWord = vectorizacion(bagOfWords, len(vocabulary), 1)
similitud = getSimilitud("organización", dicVectorWord)
writeA("/content/drive/MyDrive/PLN/Practicas/Ejercicio1/Salidas/similitudRaw.json", similitud)
print(similitud)

{'organización': 1.0, 'sala': 0.717, 'raron': 0.7, 'licitación': 0.699, 'recupere': 0.699, 'eso': 0.688, 'función': 0.685, 'desde': 0.68, 'ferrando': 0.676, 'crédito': 0.675, 'maestra': 0.668, 'equidad': 0.666, 'fugas': 0.663, 'oponerse': 0.66, 'afamada': 0.654, 'sean': 0.651, 'méritos': 0.649, 'concibió': 0.648, 'gasolinas': 0.643, 'apoyar': 0.641, 'emergente': 0.637, 'croc': 0.636, 'préstamo': 0.635, 'heridos': 0.632, 'medicina': 0.632, 'batres': 0.631, 'exactamente': 0.631, 'nacionalistas': 0.631, 'nivel': 0.63, 'cristiana': 0.629, 'efectuará': 0.629, 'contribuyó': 0.627, 'supuesto': 0.627, 'alcista': 0.625, 'ridades': 0.624, 'anota': 0.623, 'autorizaban': 0.623, 'argumentos': 0.621, 'cese': 0.62, 'encuesta': 0.62, 'epresidenta': 0.62, 'halagos': 0.62, 'emitidos': 0.619, 'obligó': 0.619, 'gabriel': 0.616, 'expresarle': 0.615, 'fraude': 0.615, 'subida': 0.615, 'periodismo': 0.613, 'región': 0.613, 'acercan': 0.612, 'diesel': 0.612, 'latinoamericanos': 0.609, 'moctezuma': 0.609, 'sóli

In [9]:
dicVectorWord = vectorizacion(bagOfWords, len(vocabulary), 2)
similitud = getSimilitud("organización", dicVectorWord)
writeA("/content/drive/MyDrive/PLN/Practicas/Ejercicio1/Salidas/similitudFrecuencia.json", similitud)
print(similitud)

{'organización': 1.0, 'sala': 0.717, 'raron': 0.7, 'licitación': 0.699, 'recupere': 0.699, 'eso': 0.688, 'función': 0.685, 'desde': 0.68, 'ferrando': 0.676, 'crédito': 0.675, 'maestra': 0.668, 'equidad': 0.666, 'fugas': 0.663, 'oponerse': 0.66, 'afamada': 0.654, 'sean': 0.651, 'méritos': 0.649, 'concibió': 0.648, 'gasolinas': 0.643, 'apoyar': 0.641, 'emergente': 0.637, 'croc': 0.636, 'préstamo': 0.635, 'heridos': 0.632, 'medicina': 0.632, 'batres': 0.631, 'exactamente': 0.631, 'nacionalistas': 0.631, 'nivel': 0.63, 'cristiana': 0.629, 'efectuará': 0.629, 'contribuyó': 0.627, 'supuesto': 0.627, 'alcista': 0.625, 'ridades': 0.624, 'anota': 0.623, 'autorizaban': 0.623, 'argumentos': 0.621, 'cese': 0.62, 'encuesta': 0.62, 'epresidenta': 0.62, 'halagos': 0.62, 'emitidos': 0.619, 'obligó': 0.619, 'gabriel': 0.616, 'expresarle': 0.615, 'fraude': 0.615, 'subida': 0.615, 'periodismo': 0.613, 'región': 0.613, 'acercan': 0.612, 'diesel': 0.612, 'latinoamericanos': 0.609, 'moctezuma': 0.609, 'sóli

In [10]:
dicVectorWord = vectorizacion(bagOfWords, len(vocabulary), 3)
similitud = getSimilitud("organización", dicVectorWord)
writeA("/content/drive/MyDrive/PLN/Practicas/Ejercicio1/Salidas/similitudTF_Sublineal.json", similitud)
print(similitud)

{'organización': 1.0, 'contienen': 0.437, 'moctezuma': 0.424, 'almazán': 0.423, 'ordenada': 0.419, 'motín': 0.414, 'peraza': 0.414, 'comprometerán': 0.41, 'aportados': 0.408, 'obligó': 0.408, 'avenidas': 0.404, 'gasolinas': 0.403, 'ortografía': 0.403, 'oponerse': 0.398, 'solicita': 0.397, 'productor': 0.396, 'extradición': 0.395, 'requisitos': 0.395, 'analiza': 0.393, 'préstamo': 0.393, 'actitudes': 0.392, 'efectuará': 0.392, 'líder': 0.392, 'ces': 0.391, 'afamada': 0.39, 'bachilleres': 0.389, 'halagos': 0.389, 'agitado': 0.388, 'rech': 0.388, 'mujer': 0.387, 'stevens': 0.387, 'empresario': 0.385, 'disputa': 0.384, 'elección': 0.384, 'sala': 0.383, 'ompleta': 0.381, 'lacónico': 0.38, 'argentinas': 0.379, 'nayarit': 0.377, 'aspiraciones': 0.376, 'conferencia': 0.376, 'interpreta': 0.376, 'lendor': 0.376, 'batres': 0.375, 'ipab': 0.375, 'modernización': 0.375, 'sean': 0.375, 'cese': 0.374, 'aunque': 0.373, 'exento': 0.373, 'bruto': 0.372, 'compr': 0.372, 'conformarán': 0.372, 'confrontac

In [11]:
dicVectorWord = vectorizacion(bagOfWords, len(vocabulary), 4)
similitud = getSimilitud("organización", dicVectorWord)
writeA("/content/drive/MyDrive/PLN/Practicas/Ejercicio1/Salidas/similitudIDF_Cos.json", similitud)
print(similitud)

{'organización': 1.0, 'motín': 0.287, 'conferencia': 0.284, 'almazán': 0.266, 'extradición': 0.254, 'contienen': 0.251, 'escrituras': 0.248, 'seno': 0.246, 'inversionistas': 0.237, 'edificios': 0.229, 'rech': 0.221, 'pérdidas': 0.22, 'moctezuma': 0.211, 'raron': 0.211, 'gasolinas': 0.206, 'retendidas': 0.206, 'cese': 0.205, 'lacónico': 0.204, 'exento': 0.202, 'sala': 0.202, 'caldera': 0.201, 'oponerse': 0.2, 'exactamente': 0.198, 'afamada': 0.197, 'productor': 0.196, 'inflación': 0.195, 'atómicas': 0.19, 'fructífera': 0.19, 'licitación': 0.19, 'mimetismo': 0.189, 'ortografía': 0.189, 'desde': 0.186, 'felicita': 0.186, 'halagos': 0.186, 'natividad': 0.186, 'heridos': 0.185, 'señala': 0.185, 'analizar': 0.183, 'salí': 0.182, 'distritales': 0.181, 'avenidas': 0.18, 'agitado': 0.179, 'mujer': 0.179, 'necesitará': 0.179, 'industria': 0.178, 'funcionar': 0.177, 'infortunada': 0.177, 'pagados': 0.176, 'selected': 0.176, 'envidiable': 0.175, 'región': 0.175, 'aunque': 0.174, 'elección': 0.174,